In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import enum
from pydantic import BaseModel, Field, field_validator
import instructor
from openai import OpenAI
from typing import Literal, Optional, List

In [32]:
import feedparser
podcast_atom_link = "https://api.substack.com/feed/podcast/1084089.rss" # latent space podcast
parsed = feedparser.parse(podcast_atom_link)
episode = [ep for ep in parsed.entries if ep['title'] == "Why AI Agents Don't Work (yet) - with Kanjun Qiu of Imbue"][0]
episode_summary = episode['summary']
print(episode_summary[:100])

<p><em>Thanks to the </em><em>over 11,000 people</em><em> who joined us for the first AI Engineer Su


In [33]:
from unstructured.partition.html import partition_html
parsed_summary = partition_html(text=''.join(episode_summary)) 
start_of_transcript = [x.text for x in parsed_summary].index("Transcript") + 1
print(f"First line of the transcript: {start_of_transcript}")
text = '\n'.join(t.text for t in parsed_summary[start_of_transcript:])
text = text[:3508] # shortening the transcript for speed & cost

First line of the transcript: 60


In [34]:
text

"Alessio: Hey everyone, welcome to the Latent Space Podcast. This is Alessio, Partner and CTO at Residence at Decibel Partners, and I'm joined by my co-host Swyx, founder of Smol.ai. [00:00:19]\nSwyx: Hey, and today in the studio we have Kanjun from Imbue. Welcome. So you and I have, I guess, crossed paths a number of times. You're formerly named Generally Intelligent and you've just announced your rename, rebrand in huge, humongous ways. So congrats on all of that. And we're here to dive in into deeper detail on Imbue. We like to introduce you on a high level basis, but then have you go into a little bit more of your personal side. So you graduated your BS at MIT and you also spent some time at the MIT Media Lab, one of the most famous, I guess, computer hacking labs in the world. Then you graduated MIT and you went straight into BizOps at Dropbox, where you're eventually chief of staff, which is a pretty interesting role we can dive into later. And then it seems like the founder bug 

## OpenAI

In [3]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

## Anthropic

In [ ]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    base_url=os.environ.get("ANTHROPIC_BASE_URL")
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-opus-20240229",
)
print(message.content)

## Instructor

### Minimal examples

In [ ]:
class Labels(str, enum.Enum):
    """Enumeration for single-label text classification."""

    SPAM = "spam"
    NOT_SPAM = "not_spam"


class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """

    class_label: Labels

In [ ]:
from openai import OpenAI
import instructor

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())


def classify(data: str) -> SinglePrediction:
    """Perform single-label classification on the input text."""
    return client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=SinglePrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following text: {data}",
            },
        ],
    )  # type: ignore

In [ ]:
# Test single-label classification
prediction = classify("Hello there I'm a Nigerian prince and I want to give you money")
assert prediction.class_label == Labels.SPAM

In [ ]:
#README

In [ ]:
# Define your desired output structure
class UserInfo(BaseModel):
    name: str
    age: int


# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

print(user_info.name)
#> John Doe
print(user_info.age)
#> 30

In [ ]:
from anthropic import Anthropic

class User(BaseModel):
    name: str
    age: int


client = instructor.from_anthropic(Anthropic())

# note that client.chat.completions.create will also work
resp = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Extract Jason is 25 years old.",
        }
    ],
    response_model=User,
)

assert isinstance(resp, User)
assert resp.name == "Jason"
assert resp.age == 25

In [ ]:
# Felix Vemmer example

In [ ]:
# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

In [ ]:
content = """

In the vast landscape of technological advancements, few innovations hold the promise of revolutionizing our world quite like quantum computing. It’s a realm where traditional binary systems morph into quantum bits or qubits, offering an unparalleled potential to tackle problems deemed insurmountable by classical computers. As we embark on this journey into the enigmatic depths of quantum computing, we’re met with a blend of excitement, curiosity, and a touch of trepidation. At the heart of quantum computing lies the principle of superposition, where qubits can exist in multiple states simultaneously, unlike classical bits, which are limited to either a 0 or 1. This fundamental property enables quantum computers to process vast amounts of information in parallel, promising exponential speed-ups for certain computations. Imagine solving complex optimization problems, simulating molecular structures with precision, or cracking cryptographic codes at an unprecedented pace – these are just glimpses of what quantum computing could offer. However, quantum computing isn’t merely a progression of classical computing; it’s a paradigm shift. Entanglement, another cornerstone of quantum mechanics, adds another layer of complexity. When qubits become entangled, the state of one qubit instantaneously influences the state of another, regardless of the distance between them. This phenomenon opens the door to secure communication channels and novel approaches to information processing. In conclusion, the journey into the enigmatic depths of quantum computing is filled with promise, challenges, and endless possibilities. It’s a journey that beckons us to push the boundaries of our understanding, to embrace uncertainty, and to dare to dream of a future where quantum computers empower humanity to solve the unsolvable. As we navigate this uncharted territory, one thing remains certain – the age of quantum computing has dawned, and the adventure has only just begun."""

In [ ]:
class Metadata(BaseModel):
    meta_title: str = Field(
        ...,
        min_length=50,
        max_length=60,
        description="Meta Title between 50-60 characters. Should be concise, descriptive and include primary keyword.",
    )
    meta_description: str = Field(
        ...,
        min_length=150,
        max_length=160,
        description="Meta Description between 150-160 characters. Should be compelling, summarize the page content and include relevant keywords naturally.",
    )


meta_descrioption = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_retries=5, # Retry the request 3 times
    response_model=Metadata,
    messages=[
        {"role": "system", "content": content},
    ],
)


In [ ]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    country_origin: str = Field(description="Country they were born in")

    @field_validator('country_origin')
    @classmethod
    def country_upper(cls, v: str) -> str:
        if not v.isupper():
            raise ValueError('Must be all caps!')
        return v


actor_query = "Generate the info for a random actor that is not from the US."

actor = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_retries=2, # Retry the request 3 times
    response_model=Actor,
    messages=[
        {"role": "system", "content": actor_query},
    ],
)
actor

### Blog example

In [ ]:
class Person(BaseModel):
    name: str
    school: Optional[str] = Field(..., description="The school this person attended")
    company: Optional[str] = Field(..., description="The company this person works for ")

class People(BaseModel):
    people: List[Person]

In [ ]:
client = instructor.from_openai(OpenAI())

In [ ]:
class Company(BaseModel):
    name:str

class ResearchPaper(BaseModel):
    paper_name:str = Field(..., description="an academic paper reference discussed")
    
class ExtractedInfo(BaseModel):
    people: List[Person]
    companies: List[Company]
    research_papers: Optional[List[ResearchPaper]]

response = client.chat.completions.create(
    model="gpt-4",
    response_model=ExtractedInfo,
    messages=[
        {"role": "user", "content": text},
    ]
)
print(response)

## Pydantic V1

In [3]:
from pydantic.v1 import BaseModel as BaseModelV1

In [4]:
# Define your desired output structure
class UserInfo(BaseModelV1):
    name: str
    age: int


# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

print(user_info.name)
#> John Doe
print(user_info.age)
#> 30

TypeError: UserInfo.__init_subclass__() takes no keyword arguments

## Summary

Very slick library that provides a very clear interface with the underlying model

PRO:
- instantiates Pydantic models with ease
- very simple and intuitive API
- support for more complex technologies
- easy support for other model vendors (not sure if this would also hold for marvin)

CON:
- only pydantic V2
- one man show, but a good one!